In [15]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# Libraries for Dataset
from torchvision import transforms
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

# Summary writer and path
from torch.utils.tensorboard import SummaryWriter

# Libraries for array operations
import numpy as np
import matplotlib.pyplot as plt

# Libraries for Pytorch dataset module
from torch.utils.data import Dataset, DataLoader

#Libraries for ML
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#Visualization and Saving Image
import matplotlib.pyplot as plt
from torchvision.utils import save_image


In [29]:
# Fetching Dataset
mnist_data = fetch_openml('mnist_784')

# Seperating data and Target
X = np.array(mnist_data.data.astype('float32'))
y = np.array(mnist_data.target.astype('int64'))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

ValueError: ignored

In [18]:
np.random.randint(100, size=1)[0]

73

In [19]:
class Mnist_Train_Dataset(Dataset):

  def __init__(self,noise=0,noise_mean = 0, noise_std = 0.2):

    '''
    Dataset module design for train dataset
    '''

    self.noise = noise

    self.noise_mean = noise_mean

    self.noise_std = noise_std

    self.size = np.shape(X_train)[1]

    self.max = 255

  def __len__(self):

    '''
    Returning number of datapoints in training
    '''

    return np.shape(X_train)[0]

  def __getitem__(self,idx):
    '''
    Return Data vector and target vector
    
    Feature Vector : Normalized pixel values (Some noise might be added)

    Target Vector : Normalized pixel values (Without noise)
    '''

    # Normalization

    feature_vector = X_train[idx]/self.max

    target_vector = X_train[idx]/self.max

    # Adding Noise to the signal

    if np.random.uniform(1,0,1)[0] > 0.5:

      noise_signal = np.random.normal(self.noise_mean, self.noise_std, self.size)

      feature_vector += noise_signal

    # Changing shape of the vector

    feature_vector = torch.squeeze(torch.from_numpy(np.reshape(feature_vector,(1,self.size)))).float()

    target_vector = torch.squeeze(torch.from_numpy(np.reshape(target_vector,(1,self.size)))).float()

    sample = {'feature_vector':feature_vector, 'target_vector':target_vector}

    return sample


In [20]:
class Mnist_Validation_Dataset(Dataset):

  def __init__(self,noise=0,noise_mean = 0, noise_std = 0.2):

    self.noise = noise

    self.noise_mean = noise_mean

    self.noise_std = noise_std

    self.size = np.shape(X_train)[1]

    self.max = 255

  def __len__(self):

    # Return Length of the total data points

    return np.shape(X_test)[0]

  def __getitem__(self,idx):
    '''
    Return Data vector and target vector
    
    Feature Vector : Normalized pixel values (Some noise might be added)

    Target Vector : Normalized pixel values (Without noise)
    '''

    # Normalization

    feature_vector = X_test[idx]/self.max

    target_vector = X_test[idx]/self.max

    # Adding Noise to the signal

    if np.random.uniform(1,0,1)[0] > 0.5:

      noise_signal = np.random.normal(self.noise_mean, self.noise_std, self.size)

      feature_vector += noise_signal

    # Changing shape of the vector
    
    feature_vector = torch.squeeze(torch.from_numpy(np.reshape(feature_vector,(1,self.size)))).float()

    target_vector = torch.squeeze(torch.from_numpy(np.reshape(target_vector,(1,self.size)))).float()

    sample = {'feature_vector':feature_vector, 'target_vector':target_vector}

    return sample


In [21]:
class Mnist_Visualize(Dataset):

  def __init__(self,noise=0,noise_mean = 0, noise_std = 0.2):

    self.noise = noise

    self.noise_mean = noise_mean

    self.noise_std = noise_std

    self.size = np.shape(X_train)[1]

    self.max = 255

    # Calculating positive and negative indices of every target
    self.similar_target = []

    for i in range(0,10):
      self.similar_target.append(np.where(y_test==i)[0])

  def __len__(self):

    # Return Length of the total data points

    return np.shape(X_test)[0]

  def __getitem__(self,idx):
    '''
    Return Data vector and target vector
    
    Feature Vector : Normalized pixel values (Some noise might be added)

    Target Vector : Normalized pixel values (Without noise)
    '''
    sample = {}

    for i in range(0,10):


      # Normalization

      feature_vector = X_test[np.random.choice(self.similar_target[i],1)[0]]/self.max

      target_vector = X_test[np.random.choice(self.similar_target[i],1)[0]]/self.max

      # Adding Noise to the signal

      if np.random.uniform(1,0,1)[0] > 0.5:

        noise_signal = np.random.normal(self.noise_mean, self.noise_std, self.size)

        feature_vector += noise_signal

      # Changing shape of the vector
      
      feature_vector = torch.squeeze(torch.from_numpy(np.reshape(feature_vector,(1,self.size)))).float()

      target_vector = torch.squeeze(torch.from_numpy(np.reshape(target_vector,(1,self.size)))).float()

      s = {'feature_vector':feature_vector, 'target_vector':target_vector}

      sample[i] = s

    return sample


In [22]:
class VAE(nn.Module):

  def __init__(self,latent_dimentions=2, p=0.2):

    super(VAE,self).__init__()

    '''
    Lets define encoder parameters.
    '''

    en_inp_dim = 28*28

    en_hid1 = 500

    en_hid2 = 300

    en_hid3 = 200

    en_out = latent_dimentions

    self.encoder1 = nn.Linear(en_inp_dim,en_hid1)

    self.en_drop1 = nn.Dropout(p=p)

    self.encoder2 = nn.Linear(en_hid1,en_hid2)

    self.en_drop2 = nn.Dropout(p=p)

    self.encoder3 = nn.Linear(en_hid2,en_hid3)

    self.en_drop3 = nn.Dropout(p=p)

    self.mean_layer = nn.Linear(en_hid3,en_out)

    self.log_var_layer = nn.Linear(en_hid3,en_out)


    '''
    Defining parameters for decoder
    '''

    dec_inp_dim = latent_dimentions

    dec_hid1 = 200

    dec_hid2 = 300

    dec_hid3 = 500

    dec_out = 28*28

    self.decoder1 = nn.Linear(dec_inp_dim,dec_hid1)

    self.dec_drop1 = nn.Dropout(p=p)

    self.decoder2 = nn.Linear(dec_hid1,dec_hid2)

    self.dec_drop2 = nn.Dropout(p=p)

    self.decoder3 = nn.Linear(dec_hid2,dec_hid3)

    self.dec_drop3 = nn.Dropout(p=p)

    self.decoder_out_layer = nn.Linear(dec_hid3,dec_out)



  def Encoder(self,x):

    '''
    Defining Dncoder Layer
    '''

    x = F.relu(self.encoder1(x))

    x = self.en_drop1(x)

    x = F.relu(self.encoder2(x))

    x = self.en_drop2(x)

    x = F.relu(self.encoder3(x))

    x = self.en_drop3(x)

    mean = self.mean_layer(x)   

    log_var = self.log_var_layer(x)

    return mean,log_var

  def Decoder(self,z):

    '''
    Defining Decoder Layer
    '''

    x = F.relu(self.decoder1(z))

    x = self.dec_drop1(x)

    x = F.relu(self.decoder2(x))

    x = self.dec_drop2(x)

    x = F.relu(self.decoder3(x))

    x = self.dec_drop3(x)

    x = F.sigmoid(self.decoder_out_layer(x))

    return x

  def Sample(self,mean,log_var):

    '''
    Sampling from gaussian ditribution using reparameterization technique
    '''
    #Satandard Deviation
    std = torch.exp(0.5*log_var)
    #Gaussian sample with Standard deviation std
    eps = torch.randn_like(std)
    # Gaussian sample with Standard deviation std and Mean mean
    return eps.mul(std).add_(mean)

  def forward(self, x):
    '''
    Forward pass through VAE
    '''
    # Extracting mean and variance from encoder
    mu, log_var = self.Encoder(x)
    # Fetching sample from distribution
    z = self.Sample(mu, log_var)
    # Return decoded signal with mean and log variance generated
    return (self.Decoder(z), mu, log_var)


In [23]:
def Negative_ELBO_BCE_KL(pred, target, mean, log_var, batch_size):
  '''
  Function that compute ELBO and negates it so that we can convert maximization task to minimization task
  '''

  # Calculating Binary Cross Entropy
  # This part is to check reconstruction of target image
  BCE = F.binary_cross_entropy(pred, target, reduction='sum') 

  # Calculating KL Divergence
  # This Calculates how good is mean and variance
  KL = -0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())

  return (BCE + KL) / batch_size



In [24]:
#Setting up the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Reading data loader and dataset class
train_data = Mnist_Train_Dataset()
train_dataloader = DataLoader(train_data,batch_size=128)

# Reading data loader and dataset class
validation_data = Mnist_Validation_Dataset()
validation_dataloader = DataLoader(validation_data,batch_size=128)

# Visualizing MNIST
mnist_visualize = Mnist_Visualize()

# Defining VAE Model 
net = VAE(latent_dimentions=2).to(device)

# Defining loss function
criterion = nn.CrossEntropyLoss()

#Optimizer selection
#optimizer = optim.SGD(net.parameters(),lr = 0.0001)
optimizer = optim.Adam(net.parameters())

In [25]:
#mnist_visualize[0]

In [26]:
# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('/content/drive/My Drive/Colab Notebooks/VAE/MNIST_VAE_NN_BCE_KL/BCE_KL_NN/LD2/')

In [27]:

for epoch in range(201):

  running_loss = 0
  running_acc = 0
  count = 0

  for i,data in enumerate(train_dataloader):
    net.train()
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data['feature_vector'].to(device),data['target_vector'].to(device)

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs)
    
    #Calculating Accuracy
    #accuracy = torch.sum(torch.argmax(outputs,dim=1) == labels).float()/np.shape(labels)[0]
    #print(outputs.size())

    #Calculating Loss
    loss = Negative_ELBO_BCE_KL(outputs[0], labels, outputs[1], outputs[2], np.shape(labels)[0]) 
    loss.backward()
    optimizer.step()

    

    # print statistics
    running_loss += loss.item()
    #running_acc  += accuracy.item()
    count += 1
    if i % 100 == 0:    # print every 100 mini-batches
      #After one training loop
      print('------------------------------------------------------------------------------------------------------')
      print('Epoch:',epoch+1)
      print('Iteration: %d Average Training Loss: %.3f' % (i, running_loss / count))
      writer.add_scalar('Average Training Loss : ',running_loss / count,epoch * len(train_dataloader) + i)
      
      running_loss = 0.0
      
      count = 0
      val_loss = 0
      val_acc = 0
      val_count = 0
      net.eval()
      with torch.no_grad():
        for val_i,val_data in enumerate(validation_dataloader):
          val_feature, val_labels = val_data['feature_vector'].to(device),val_data['target_vector'].to(device)
          val_outputs = net(val_feature)
          val_loss += Negative_ELBO_BCE_KL(val_outputs[0], val_labels, val_outputs[1], val_outputs[2], np.shape(val_labels)[0]).item()
          #val_acc += (torch.sum(torch.argmax(val_outputs,dim=1) == val_labels).float()/np.shape(val_labels)[0]).item()
          val_count += 1
          #if val_i%10000 == 0:
          #  break
          
      print('Average Validation Loss:',val_loss/val_count)
      writer.add_scalar('Average Validation Loss',val_loss / val_count,epoch * len(train_dataloader) + i)
      
  if epoch % 50 == 0:

    # Every 50 epochs we save sample images

    temp_full = mnist_visualize[0]

    for key,val in temp_full.items():

      out = net(val['feature_vector'].to(device))

      save_image(out[0].data.view(28, 28),'/content/drive/My Drive/Colab Notebooks/VAE/MNIST_VAE_NN_BCE_KL/BCE_KL_NN/LD2/' + str(epoch)+'/'+str(key)+'_pred' + '.png')
      save_image(val['target_vector'].view(28,28),'/content/drive/My Drive/Colab Notebooks/VAE/MNIST_VAE_NN_BCE_KL/BCE_KL_NN/LD2/' + str(epoch)+'/'+str(key)+'.png')



/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


------------------------------------------------------------------------------------------------------
Epoch: 1
Iteration: 0 Average Training Loss: 543.187
Average Validation Loss: 532.6181498061131
------------------------------------------------------------------------------------------------------
Epoch: 1
Iteration: 100 Average Training Loss: 226.484
Average Validation Loss: 194.44075479994726
------------------------------------------------------------------------------------------------------
Epoch: 1
Iteration: 200 Average Training Loss: 192.504
Average Validation Loss: 187.81905092114079
------------------------------------------------------------------------------------------------------
Epoch: 1
Iteration: 300 Average Training Loss: 188.827
Average Validation Loss: 184.2588553324233
------------------------------------------------------------------------------------------------------
Epoch: 1
Iteration: 400 Average Training Loss: 183.768
Average Validation Loss: 175.949707699

In [28]:
# Closing the Writer and saving the model. Saving the model is not enough, we also need to save the model class as well
writer.close()
PATH = '/content/drive/My Drive/Colab Notebooks/VAE/MNIST_VAE_NN_BCE_KL/BCE_KL_NN/LD2/saved_model'
torch.save(net.state_dict(), PATH)